# 3. LangChain Streaming - Respuestas en Tiempo Real

## Objetivos de Aprendizaje
- Comprender qué es el streaming y cuándo usarlo
- Implementar streaming con LangChain
- Manejar chunks de datos en tiempo real
- Construir interfaces de usuario reactivas

## ¿Qué es el Streaming?

El streaming permite recibir la respuesta del modelo **token por token** conforme se genera, en lugar de esperar a que termine completamente. Esto mejora significativamente la experiencia de usuario en aplicaciones interactivas.

### Ventajas del Streaming:
- **Percepción de velocidad**: El usuario ve progreso inmediato
- **Mejor UX**: Interfaces más reactivas e interactivas  
- **Engagement**: Mantiene la atención del usuario
- **Debugging**: Permite ver el proceso de generación

### Casos de Uso Ideales:
- Chatbots y asistentes conversacionales
- Generación de contenido largo
- Aplicaciones web interactivas
- Demostraciones en vivo

In [ ]:
# Importar bibliotecas necesarias
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import os
import time

print("Bibliotecas importadas correctamente para streaming")

In [ ]:
# Configuración del modelo con streaming habilitado
try:
    llm = ChatOpenAI(
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("GITHUB_TOKEN"),
        model="gpt-4.1",
        streaming=True,  # ¡Importante: habilitar streaming!
        temperature=0.7
    )
    
    print("✓ Modelo configurado con streaming habilitado")
    print(f"Modelo: {llm.model_name}")
    print(f"Streaming: {llm.streaming}")
    
except Exception as e:
    print(f"✗ Error en configuración: {e}")
    print("Verifica las variables de entorno")

## Streaming Básico

El método `.stream()` devuelve un generador que produce chunks de texto conforme se generan.

In [ ]:
# Ejemplo básico de streaming
def streaming_basico():
    prompt = "Cuéntame una historia corta sobre un programador que descubre la magia en el código"
    
    print("=== STREAMING EN TIEMPO REAL ===")
    print("Generando respuesta...")
    print("-" * 50)
    
    try:
        # stream() devuelve un generador de chunks
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            # Imprimir cada chunk sin nueva línea
            print(chunk.content, end="", flush=True)
            time.sleep(0.01)  # Pequeña pausa para simular streaming visual
            
        print("\n" + "-" * 50)
        print("✓ Streaming completado")
        
    except Exception as e:
        print(f"✗ Error en streaming: {e}")

# Ejecutar streaming básico
streaming_basico()

## Comparación: Streaming vs No-Streaming

Veamos la diferencia en experiencia de usuario entre ambos enfoques.

In [ ]:
# Comparación entre streaming y no-streaming
def comparar_streaming():
    # Modelo sin streaming
    llm_no_stream = ChatOpenAI(
        base_url=os.getenv("OPENAI_BASE_URL"),
        api_key=os.getenv("GITHUB_TOKEN"),
        model="gpt-4o",
        streaming=False,  # Sin streaming
        temperature=0.7
    )
    
    prompt = "Escribe un párrafo sobre las ventajas de la programación en Python"
    
    print("=== COMPARACIÓN: STREAMING vs NO-STREAMING ===\\n")
    
    # 1. Sin streaming
    print("1. SIN STREAMING:")
    print("-" * 20)
    print("Esperando respuesta completa...")
    
    start_time = time.time()
    try:
        response = llm_no_stream.invoke([HumanMessage(content=prompt)])
        end_time = time.time()
        
        print(f"\\n[Respuesta recibida después de {end_time - start_time:.2f} segundos]")
        print(response.content)
        
    except Exception as e:
        print(f"Error: {e}")
    
    print("\\n" + "="*60 + "\\n")
    
    # 2. Con streaming
    print("2. CON STREAMING:")
    print("-" * 18)
    print("Respuesta en tiempo real:")
    
    start_time = time.time()
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            print(chunk.content, end="", flush=True)
            time.sleep(0.03)  # Simular pausa para efecto visual
        
        end_time = time.time()
        print(f"\\n\\n[Streaming completado en {end_time - start_time:.2f} segundos]")
        
    except Exception as e:
        print(f"Error: {e}")
    
    print("\\n" + "="*60)
    print("OBSERVACIONES:")
    print("- Sin streaming: El usuario espera sin feedback")
    print("- Con streaming: El usuario ve progreso inmediato")
    print("- Mejor percepción de velocidad con streaming")
    print("- Streaming es especial para respuestas largas")

# Ejecutar comparación
comparar_streaming()

## Implementación de un Chatbot Simple con Streaming

Creemos un chatbot básico que demuestre el streaming en un contexto práctico.

In [ ]:
# Chatbot simple con streaming
def chatbot_streaming():
    print("=== CHATBOT CON STREAMING ===")
    print("Escribe 'salir' para terminar la conversación\\n")
    
    # Configurar asistente con personalidad
    system_message = """Eres un asistente útil y amigable especializado en tecnología. 
    Respondes de manera clara y concisa, y siempre intentas ser educativo."""
    
    while True:
        # Obtener input del usuario
        user_input = input("\\n🧑 Tú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("\\n👋 ¡Hasta luego!")
            break
            
        if not user_input.strip():
            continue
            
        print("\\n🤖 Asistente: ", end="", flush=True)
        
        try:
            # Streaming de la respuesta
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_input}
            ]
            
            # Convertir a formato LangChain
            from langchain.schema import SystemMessage
            lc_messages = [
                SystemMessage(content=system_message),
                HumanMessage(content=user_input)
            ]
            
            full_response = ""
            for chunk in llm.stream(lc_messages):
                content = chunk.content
                print(content, end="", flush=True)
                full_response += content
                time.sleep(0.02)
                
            print()  # Nueva línea al final
            
        except KeyboardInterrupt:
            print("\\n\\n⏸️ Interrumpido por el usuario")
            break
        except Exception as e:
            print(f"\\n❌ Error: {e}")
            
    print("\\n¡Gracias por usar el chatbot!")

# Ejecutar chatbot (¡Pruébalo!)
chatbot_streaming()  # Descomenta esta línea para ejecutar

print("💡 Descomenta la línea anterior para probar el chatbot interactivo")

## Streaming Avanzado con Manejo de Chunks

Podemos procesar cada chunk individualmente para crear experiencias más sofisticadas.

In [ ]:
# Streaming con análisis de chunks
def streaming_avanzado():
    prompt = "Explica qué es la inteligencia artificial y cómo funciona el machine learning"
    
    print("=== STREAMING AVANZADO CON ANÁLISIS ===")
    print("Analizando chunks conforme llegan...\n")
    
    # Variables para estadísticas
    chunk_count = 0
    total_content = ""
    words_processed = 0
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            chunk_count += 1
            content = chunk.content
            total_content += content
            
            # Contar palabras aproximadas
            if content.strip():
                words_in_chunk = len(content.split())
                words_processed += words_in_chunk
            
            # Mostrar progreso cada 10 chunks
            if chunk_count % 10 == 0:
                print(f"\\n[Progreso: {chunk_count} chunks, ~{words_processed} palabras]\\n")
            
            # Imprimir el contenido
            print(content, end="", flush=True)
            time.sleep(0.02)  # Pausa ligeramente más larga para ver el análisis
        
        # Estadísticas finales
        print(f"\\n\\n=== ESTADÍSTICAS FINALES ===")
        print(f"Total de chunks: {chunk_count}")
        print(f"Palabras aproximadas: {words_processed}")
        print(f"Caracteres totales: {len(total_content)}")
        print(f"Promedio chars/chunk: {len(total_content)/chunk_count if chunk_count > 0 else 0:.1f}")
        
    except Exception as e:
        print(f"\\n✗ Error: {e}")

# Ejecutar streaming avanzado
streaming_avanzado()

## Consideraciones Técnicas del Streaming

### Cuándo Usar Streaming:
✅ **SÍ usar streaming:**
- Respuestas largas (>100 tokens)
- Aplicaciones interactivas
- Chatbots y asistentes
- Demostraciones en vivo
- Cuando la UX es prioritaria

❌ **NO usar streaming:**
- Respuestas muy cortas
- Procesamiento batch
- APIs de backend sin interfaz
- Cuando necesitas la respuesta completa antes de procesar

### Mejores Prácticas:
1. **Manejo de errores**: Siempre incluye try/catch
2. **Indicadores visuales**: Muestra progreso al usuario
3. **Cancelación**: Permite al usuario interrumpir
4. **Buffer management**: Para interfaces web, considera buffering
5. **Performance**: Monitorea el uso de recursos

## Ejercicios Prácticos

### Ejercicio 1: Indicador de Progreso
Modifica el código para mostrar un indicador de progreso (spinner, barra, porcentaje).

### Ejercicio 2: Streaming con Filtros
Implementa streaming que filtre o procese chunks específicos (ej: resaltar palabras clave).

### Ejercicio 3: Chatbot Mejorado
Extiende el chatbot con:
- Historial de conversación
- Comandos especiales (/help, /clear)
- Diferentes personalidades

## Conceptos Clave Aprendidos

1. **Streaming** mejora la percepción de velocidad
2. **Chunks** se procesan individualmente en tiempo real
3. **UX** es significativamente mejor con streaming
4. **Implementación** requiere manejo cuidadoso de generadores
5. **Casos de uso** específicos donde streaming aporta valor

## Próximos Pasos

En el siguiente notebook exploraremos la **memoria en LangChain**, que nos permite mantener contexto entre múltiples interacciones del usuario.